In [6]:
import networkx as nx
import numpy as np
import json

In [7]:
def createPreferenceGraph(n, maxPreferencesPerCategory) :
    """creates a Preference Graph for students' preferences (opinions & thoughts)

    Parameters
    ----------
    n : int
        Number of students in a class
    maxPreferencesPerCategory : int
        maximum possible number of edges (student's thoughts) per opinion item

    Raises
    ------
    ValueError
        If n is less than 3 or if maxPreferencesPerCategory is not between 1 and n-2.
        
    Returns
    -------
    MultiDiGraph object
        a directed graph object with allowance of multiple edges between a pair of nodes
    """

    if n < 3 :
        raise ValueError("n must be ≥ 3.")
    if maxPreferencesPerCategory < 1 or maxPreferencesPerCategory > n-2 :
        raise ValueError(f"maxPreferencesPerCategory must be 1 ≤ maxPreferencesPerCategory ≤ {n-2}")
    
    # opinion for socialization : items 0, 4
    # opinion for collaboration : items 1, 5
    # thought for perception of socialization : items 2, 6
    # thought for perception of collaboration : items 3, 7
    # define weights of edges as strings
    opinions = ["prefers to socialize with", # item = 0
                "prefers to collaborate with", # item = 1
                "perceives that the node ? prefers to socialize with him", # item = 2
                "perceives that the node ? prefers to collaborate with him", # item = 3
                "avoids socializing with", # item = 4
                "avoids collaborating with", # item = 5
                "perceives that the node ? avoids socializing with him", # item = 6
                "perceives that the node ? avoids collaborating with him"] # item = 7
    
    def editString(s, node1, node2) :
        if "?" in s :
            return f"node {node1} {str(node2).join(s.split('?'))}"
        else :
            return f"node {node1} {s} node {node2}"
        
    

    # MultiDiGraph: many directed edges between a pair of nodes
    G = nx.MultiDiGraph()
    G.add_nodes_from(range(n))

    for node in G.nodes() :
        # neglect current node from target nodes
        nodes = list(filter(lambda element : element != node, G.nodes()))

        # find number of outcoming edges for each weight item
        numberOfChoices = []
        while True:
            numberOfChoices = np.random.choice(range(1, maxPreferencesPerCategory+1), size=8)
            # each weight item has a conflicting item (preference vs avoidance)
            # each conflicting pair of weight items must have at most n-1 edges in total
            if numberOfChoices[0] + numberOfChoices[4] <= n - 1 and \
                numberOfChoices[1] + numberOfChoices[5] <= n - 1 and \
                numberOfChoices[2] + numberOfChoices[6] <= n - 1 and \
                numberOfChoices[3] + numberOfChoices[7] <= n - 1 :
                break
        
        
        # find nodes (peers) for weight items 0, 1, 2, 3
        peersA, peersB, peersC, peersD = \
            np.random.choice(nodes, size=numberOfChoices[0], replace=False), \
                np.random.choice(nodes, size=numberOfChoices[1], replace=False), \
                    np.random.choice(nodes, size=numberOfChoices[2], replace=False), \
                        np.random.choice(nodes, size=numberOfChoices[3], replace=False) 
         
        # find nodes (peers) for weight items 4, 5, 6, 7
        # !! neglect nodes already chosen in items 0, 1, 2, 3 respectively (preference vs avoidance) !!
        peersE, peersF, peersG, peersH = \
            np.random.choice(list(set(nodes)-set(peersA)), size=numberOfChoices[4], replace=False), \
                np.random.choice(list(set(nodes)-set(peersB)), size=numberOfChoices[5], replace=False), \
                    np.random.choice(list(set(nodes)-set(peersC)), size=numberOfChoices[6], replace=False), \
                        np.random.choice(list(set(nodes)-set(peersD)), size=numberOfChoices[7], replace=False)

        # add edges in preferenceGraph with a weight item and weight item index
        weightIndex = 0
        for peers in [peersA, peersB, peersC, peersD, peersE, peersF, peersG, peersH] :
            for peer in peers :
                G.add_edge(int(node), int(peer), opinion=editString(opinions[weightIndex], node, peer), opinionItem=weightIndex)
            weightIndex += 1

    return G

In [8]:
def saveGraphJson(G, path) :
    def convertToJSON(G) :
        nodes = [{"id": node} for node in G.nodes()]
        
        edges = [{
            "source": i,
            "target": j,
            "opinion": data["opinion"],
            "opinion item": data['opinionItem']
        } for i, j, data in G.edges(data=True)]

        return {
            "nodes": nodes,
            "edges": edges
        }
    with open(path, "w") as write_file:
        json.dump(convertToJSON(G), write_file)


In [9]:
options = {
    "n" : 100, # number of students
    "maxPreferencesPerCategory" : 10 # maximum possible number of edges (student's thoughts) per opinion item
}

# create preference graph
G = createPreferenceGraph(**options)

In [10]:
saveGraphJson(G, "./preferenceGraph.json")